In [23]:
import selenium
from selenium import webdriver
import os
import time
import requests
from PIL import Image
import io
import hashlib

# Put the path for your ChromeDriver here
DRIVER_PATH = 'chromedriver'

#wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [24]:
#wd.get('https://google.com')

In [25]:
#search_box = wd.find_element_by_css_selector('input.gLFyf')
#search_box.send_keys('Ship')

In [26]:
#wd.quit()

In [38]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [39]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [40]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [41]:
search_term = 'Ship'

search_and_download(search_term=search_term, driver_path=DRIVER_PATH, number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTefSOQnD9JakqoAgSFYZMJ6v5JmrGQPT2r0Lff2LOvUvf0pDOJ&usqp=CAU - as ./images\ship\ce22210c91.jpg
SUCCESS - saved https://www.aljazeera.com/mritems/imagecache/mbdxxlarge/mritems/Images/2019/8/8/3a015da06a80498eaacc2bb9fb423407_18.jpg - as ./images\ship\08f5078063.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTZgS8n7V6eL7L2lmFfNCJlHKYJJUyA4X4gbYXdxMdEgK_BrGPy&usqp=CAU - as ./images\ship\130d7e95ae.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRWTDwHPc7CrzBzhnYycjyLBRVzKNiTO_GoXOCrhpwHrVLgO4Kw&usqp=CAU - as ./images\ship\8b983b0e87.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS21OvhUOLhmbIu-h_W7DMMxcSjLyS6pb2R0GZB0pjrtL0Gthsq&usqp=CAU - as ./images\ship\a402bd3674.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRtgCsCcIx

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSolfmsKpYa_sJ1TkNpzj-1oFPnLdVzUy2Xy-NQe2qSwOQqYhcs&usqp=CAU - as ./images\ship\95137c892b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRg3Kuv4Yaz19fzz0rtkd3WqIqMWud-ANdOlm8-ldEOE0JALtZL&usqp=CAU - as ./images\ship\3770c62fd8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTkB4wlxCYz4qDTsq-oNTTihZ0Vs4Ro6CWId-gG6WgNbivMkP13&usqp=CAU - as ./images\ship\282829a29a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTQ6UCr6HafZ9KMux1ZqePDOMi0YyWRaBxkSeXO6D2BCxkoTynA&usqp=CAU - as ./images\ship\29a59abc34.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSGHqrY93XcoEqqCxmc9D9Os8kWKxa7DA0ZB-dQpKx8n9dxR1i1&usqp=CAU - as ./images\ship\2369a9864b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQNXuRHXN-u9sjkFmI7ihOEu8zSQyPbJMNxI2ZrdNr7IOSkqm84&usqp=CAU - as ./images\ship\c4b7b4a79d.jpg
SUCC